In [1]:
import json

# Open the JSON file and load the data
with open('allDocument.json', 'r') as file:
    data = json.load(file)

# Print the loaded data
# print(len(data[0]))
data[0]


{'question': "In the case of Nasr v NRMA Insurance [2006] NSWSC 1018, why was the plaintiff's appeal lodged out of time?",
 'text': "In Nasr v NRMA Insurance [2006] NSWSC 1018, the plaintiff's appeal was lodged out of time because the summons was filed on 8 June 2006, seven months after the decision of the Local Court was made on 4 October 2005. No explanation was provided for this delay.",
 'doc_id': '81ceb3c0'}